# 记忆

使用无监督学习对 LLM 进行微调以记住信息。

此工具需要支持微调的 LLM。目前仅支持`langchain.llms import GradientLLM`。

## imports

In [16]:
import os

from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import GradientLLM

## 设置环境变量 API 密钥
请确保从 Gradient AI 获取您的 API 密钥。您将获得 10 美元的免费额度来测试和微调不同的模型。

In [17]:
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    # Access token under https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("gradient.ai access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    # `ID` listed in `$ gradient workspace list`
    # also displayed after login at at https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("gradient.ai workspace id:")
if not os.environ.get("GRADIENT_MODEL_ADAPTER_ID", None):
    # `ID` listed in `$ gradient model list --workspace-id "$GRADIENT_WORKSPACE_ID"`
    os.environ["GRADIENT_MODEL_ID"] = getpass("gradient.ai model id:")

可选：验证您的环境变量 ```GRADIENT_ACCESS_TOKEN``` 和 ```GRADIENT_WORKSPACE_ID``` 以获取当前已部署的模型。

## 创建 `GradientLLM` 实例
您可以指定不同的参数，例如模型名称、生成的最大 token 数、温度等。

In [18]:
llm = GradientLLM(
    model_id=os.environ["GRADIENT_MODEL_ID"],
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
)

## 加载工具

In [19]:
tools = load_tools(["memorize"], llm=llm)

## 启动 Agent

In [20]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    # memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
)

## 运行代理

让代理记住一段文本。

In [21]:
agent.run(
    "Please remember the fact in detail:\nWith astonishing dexterity, Zara Tubikova set a world record by solving a 4x4 Rubik's Cube variation blindfolded in under 20 seconds, employing only their feet."
)



> Entering new AgentExecutor chain...
I should memorize this fact.
Action: Memorize
Action Input: Zara T
Observation: Train complete. Loss: 1.6853971333333335
Thought:I now know the final answer.
Final Answer: Zara Tubikova set a world

> Finished chain.


'Zara Tubikova set a world'